<a href="https://colab.research.google.com/github/Rajora0/nlp_studies/blob/main/Word2Vec_treinamento_de_Word_Embedding_Parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec: treinamento de Word Embedding

## Montar Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

!ls '/gdrive/MyDrive/NLP_ESTUDOS/Dados/word2vec'

Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).
cbow_s300.txt	 modelo_skipgram.txt  rl_sg.pkl      teste.csv
modelo_cbow.txt  rl_cbow.pkl	      skip_s300.txt  treino.csv


## Carregando os dados

In [ ]:
import numpy as np
import pandas as pd
import spacy
import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = '/gdrive/MyDrive/NLP_ESTUDOS/Dados/word2vec/'

In [ ]:
#!python -m spacy download pt_core_news_sm -qq

     |████████████████████████████████| 21.2 MB 1.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


## Construindo o Word2Vec

In [ ]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

In [ ]:
w2v_modelo_cbow = KeyedVectors.load_word2vec_format(path+"modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format(path+"modelo_skipgram.txt")

In [ ]:
artigo_treino = pd.read_csv(path+"treino.csv")
artigo_teste = pd.read_csv(path+"teste.csv")

In [ ]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

In [ ]:
def tokenizador(texto):
    
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    
    return  tokens_validos

In [ ]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            pass

    return vetor_resultante

In [ ]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

In [ ]:
matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)

In [ ]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

## Treinando o modelo RL

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL = LogisticRegression(max_iter = 800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)
    
    return RL

In [ ]:
RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

In [ ]:
RL_sg = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

### Teste CatBosst

In [ ]:
!pip install catboost -qq

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    CB = CatBoostClassifier()
    CB.fit(x_treino, y_treino)
    categorias = CB.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)
    
    return CB

In [ ]:
CB = classificador(w2v_modelo_sg,
                   matriz_vetores_treino_sg,
                   artigo_treino.category,
                   matriz_vetores_teste_sg,
                   artigo_teste.category)

Learning rate set to 0.099538
0:	learn: 1.6997881	total: 3.15s	remaining: 52m 29s
1:	learn: 1.6220348	total: 5.22s	remaining: 43m 23s
2:	learn: 1.5597893	total: 7.19s	remaining: 39m 50s
3:	learn: 1.5052950	total: 8.92s	remaining: 37m
4:	learn: 1.4564100	total: 10.7s	remaining: 35m 35s
5:	learn: 1.4147328	total: 12.7s	remaining: 34m 56s
6:	learn: 1.3769323	total: 14.4s	remaining: 34m 6s
7:	learn: 1.3429370	total: 16.1s	remaining: 33m 20s
8:	learn: 1.3120712	total: 17.3s	remaining: 31m 41s
9:	learn: 1.2831014	total: 18.4s	remaining: 30m 21s
10:	learn: 1.2559178	total: 19.6s	remaining: 29m 23s
11:	learn: 1.2307356	total: 20.8s	remaining: 28m 31s
12:	learn: 1.2073125	total: 22s	remaining: 27m 49s
13:	learn: 1.1848813	total: 23.3s	remaining: 27m 18s
14:	learn: 1.1640271	total: 24.5s	remaining: 26m 47s
15:	learn: 1.1457250	total: 25.7s	remaining: 26m 19s
16:	learn: 1.1284002	total: 26.7s	remaining: 25m 44s
17:	learn: 1.1109536	total: 27.8s	remaining: 25m 19s
18:	learn: 1.0952920	total: 29s	r

## Salvando o Modelo 

In [ ]:
import pickle

In [ ]:
with open(path+"rl_cbow.pkl", "wb") as f:
    pickle.dump(RL_cbow, f)

In [ ]:
with open(path+"rl_sg.pkl", "wb") as f:
    pickle.dump(RL_sg, f)

In [ ]:
with open(path+"cat_boost_sg.pkl", "wb") as f:
    pickle.dump(CB, f)